In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
display(running_order.shape)
running_order.head()


(6759, 20)

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,1,Stora Tuna OK 1,Stora Tuna OK,SWE,1,6.0,tilda östberg,1,Tilda Östberg,1.0,1.8857,0.0000,7.252938,1.860841,2.083642,0.111400,0.111400,1.8857,0.111400
1,1,1,Stora Tuna OK 1,Stora Tuna OK,SWE,2,5.7,magdalena olsson,2,Magdalena Olsson,7.0,1.9587,0.0824,6.718492,1.814416,2.009966,0.097775,0.097775,1.9587,0.082400
2,2,1,Stora Tuna OK 1,Stora Tuna OK,SWE,3,7.3,julia jakob,3,Julia Jakob,1.0,1.7824,0.0000,7.057429,1.796061,2.104793,0.154366,0.154366,1.7824,0.154366
3,3,1,Stora Tuna OK 1,Stora Tuna OK,SWE,4,7.9,tove alexandersson,4,Tove Alexandersson,7.0,1.7963,0.0565,6.395697,1.799499,1.991288,0.095895,0.095895,1.7963,0.067659
4,4,2,Göteborg Majorna OK 1,Göteborg Majorna OK,SWE,1,6.0,amanda hansson,1,Amanda Hansson,2.0,2.0053,0.0358,6.766972,1.859515,2.032438,0.086461,0.086461,2.0053,0.086461


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std", "num_runs"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std,num_runs
0,1,Stora Tuna OK 1,1,6.0,Tilda Östberg,1.8857,0.111400,1.0
1,1,Stora Tuna OK 1,2,5.7,Magdalena Olsson,1.9587,0.082400,7.0
2,1,Stora Tuna OK 1,3,7.3,Julia Jakob,1.7824,0.154366,1.0
3,1,Stora Tuna OK 1,4,7.9,Tove Alexandersson,1.7963,0.067659,7.0
4,2,Göteborg Majorna OK 1,1,6.0,Amanda Hansson,2.0053,0.086461,2.0


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                                \
leg                          1                      2                      3   
team_id                                                                        
1              Stora Tuna OK 1        Stora Tuna OK 1        Stora Tuna OK 1   
2        Göteborg Majorna OK 1  Göteborg Majorna OK 1  Göteborg Majorna OK 1   
3                   MS Parma 1             MS Parma 1             MS Parma 1   
4          Järla Orientering 1    Järla Orientering 1    Järla Orientering 1   
5               Alfta ÖSA OK 1         Alfta ÖSA OK 1         Alfta ÖSA OK 1   

                               dist                               name  \
leg                          4    1    2    3    4                   1   
team_id                                                                  
1              Stora Tuna OK 1  6.0  5.7  7.3  7.9       Tilda Östberg   
2        Göteborg Majorna OK 1  6.0  5.7  7.3  7.9      Amanda Hansson   
3                   MS Parma 1  6.0  5.7  7.3  7.9   Anastasia Rudnaya   
4          Järla Orientering 1  6.0  5.7  7.3  7.9       Sara Sjökvist   
5               Alfta ÖSA OK 1  6.0  5.7  7.3  7.9  Galina Vinogradova   

                              ... log_mean           log_std            \
leg                        2  ...        3       4         1         2   
team_id                       ...                                        
1           Magdalena Olsson  ...   1.7824  1.7963  0.111400  0.082400   
2              Runa Fremstad  ...   1.9581  1.9854  0.086461  0.087900   
3             Anni Heikkonen  ...   1.8669  1.9113  0.067659  0.100600   
4        Anna Dahlgren-Rosén  ...   1.8086  1.9289  0.112800  0.182322   
5            Josefine Heikka  ...   1.9000  1.8720  0.102900  0.094482   

                            num_runs                 
leg             3         4        1    2    3    4  
team_id                                              
1        0.154366  0.067659      1.0  7.0  1.0  7.0  
2        0.067659  0.086115      2.0  5.0  5.0  3.0  
3        0.141629  0.067659      4.0  7.0  1.0  8.0  
4        0.135658  0.108649      4.0  0.0  1.0  2.0  
5        0.095300  0.128324      7.0  2.0  6.0  3.0  

[5 rows x 24 columns]

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(1697, 24)

(1682, 24)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [8]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [9]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,31.636816,69.905465,107.965533,154.509848,49.461939,92.016380,142.714913,191.586508,39.470136,80.036666,...,32.329384,41.703044,49.461939,47.690714,58.634488,54.344031,39.470136,40.396161,43.472333,47.621385
2,37.426607,77.331566,127.278296,181.516253,52.668500,98.475909,152.161247,213.523992,44.560018,87.206440,...,45.264023,48.690798,52.668500,50.518334,58.919847,67.930557,44.560018,42.454931,51.689017,57.493089
3,34.817716,70.671761,112.248863,163.957496,45.403522,89.852030,144.791047,199.909101,39.708543,79.523564,...,35.512484,46.617247,45.403522,48.522061,62.236444,61.056161,39.708543,39.707539,47.348169,53.413566
4,34.819114,69.316597,110.521837,162.109873,53.873856,105.106089,153.021068,210.867419,43.272560,84.834634,...,34.143312,43.973979,53.873856,59.031919,57.944136,67.643390,43.272560,41.209469,44.511096,54.357056
5,32.404335,67.784394,113.545552,160.405948,48.610166,89.279655,141.547244,198.606371,39.659170,77.543103,...,40.469558,40.102259,48.610166,45.385450,58.929048,66.214170,39.659170,37.780310,48.871596,51.440843


In [10]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4,num_runs_1,num_runs_2,num_runs_3,num_runs_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,1.78,1.80,0.11,0.08,0.15,0.07,1.0,7.0,1.0,7.0
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,1.96,1.99,0.09,0.09,0.07,0.09,2.0,5.0,5.0,3.0
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,1.87,1.91,0.07,0.10,0.14,0.07,4.0,7.0,1.0,8.0
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,1.81,1.93,0.11,0.18,0.14,0.11,4.0,0.0,1.0,2.0
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,1.90,1.87,0.10,0.09,0.10,0.13,7.0,2.0,6.0,3.0


In [11]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [12]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,32.329384,41.703044,49.461939,47.690714,58.634488,54.344031,39.470136,40.396161,43.472333,47.621385
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,45.264023,48.690798,52.668500,50.518334,58.919847,67.930557,44.560018,42.454931,51.689017,57.493089
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,35.512484,46.617247,45.403522,48.522061,62.236444,61.056161,39.708543,39.707539,47.348169,53.413566
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,34.143312,43.973979,53.873856,59.031919,57.944136,67.643390,43.272560,41.209469,44.511096,54.357056
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,40.469558,40.102259,48.610166,45.385450,58.929048,66.214170,39.659170,37.780310,48.871596,51.440843


In [13]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9587   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.7963   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   
        2    Göteborg Majorna OK 1   5.7       Runa Fremstad    2.0081   
        3    Göteborg Majorna OK 1   7.3    Kristine Berglia    1.9581   

              log_std  num_runs  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
1       1    0.111400       1.0    31.636816   49.461939   39.470136   
        2    0.082400       7.0    69.905465   92.016380   80.036666   
        3    0.154366       1.0   107.965533  142.714913  123.861915   
        4    0.067659       7.0   154.509848  191.586508  171.610988   
2       1    0.086461       2.0    37.426607   52.668500   44.560018   
        2    0.087900       5.0    77.331566   98.475909   87.206440   
        3    0.067659       5.0   127.278296  152.161247  138.915991   

               fin_mean  fin_sum_log_mean  fin_sum_log_std  mass_start  \
team_id leg                                                              
1       1     39.761970          3.676499         0.113182         0.0   
        2     80.291626          4.383203         0.070141         0.0   
        3    124.289306          4.820083         0.071094         0.0   
        4    172.004992          5.146014         0.054918         0.0   
2       1     44.717357          3.796537         0.087510         0.0   
        2     87.338605          4.467877         0.061899         0.0   
        3    139.140718          4.934431         0.045928         0.0   

             dark_during_leg  personal_start95  personal_end95  \
team_id leg                                                      
1       1                0.0         31.636816       49.461939   
        2                0.0         34.428126       47.690714   
        3                0.0         32.329384       58.634488   
        4                0.0         41.703044       54.344031   
2       1                0.0         37.426607       52.668500   
        2                0.0         35.775058       50.518334   
        3                0.0         45.264023       58.919847   

             personal_median  
team_id leg                   
1       1          39.470136  
        2          40.396161  
        3          43.472333  
        4          47.621385  
2       1          44.560018  
        2          42.454931  
        3          51.689017

In [14]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [15]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]
local_start_ts = pd.Timestamp(start_timestamp).tz_localize(None)

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= local_start_ts)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= local_start_ts)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= local_start_ts)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9587   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.7963   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   

              log_std  num_runs  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
1       1    0.111400       1.0    31.636816   49.461939   39.470136   
        2    0.082400       7.0    69.905465   92.016380   80.036666   
        3    0.154366       1.0   107.965533  142.714913  123.861915   
        4    0.067659       7.0   154.509848  191.586508  171.610988   
2       1    0.086461       2.0    37.426607   52.668500   44.560018   

               fin_mean  ...  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg              ...                                                 
1       1     39.761970  ...         0.113182         0.0              0.0   
        2     80.291626  ...         0.070141         0.0              0.0   
        3    124.289306  ...         0.071094         0.0              0.0   
        4    172.004992  ...         0.054918         0.0              0.0   
2       1     44.717357  ...         0.087510         0.0              0.0   

             personal_start95  personal_end95  personal_median  \
team_id leg                                                      
1       1           31.636816       49.461939        39.470136   
        2           34.428126       47.690714        40.396161   
        3           32.329384       58.634488        43.472333   
        4           41.703044       54.344031        47.621385   
2       1           37.426607       52.668500        44.560018   

                          fin_time_median              fin_time_start95  \
team_id leg                                                               
1       1   2019-06-15 14:39:28.208155870 2019-06-15 14:31:38.208979130   
        2   2019-06-15 15:20:02.199968576 2019-06-15 15:09:54.327903748   
        3   2019-06-15 16:03:51.714893579 2019-06-15 15:47:57.932002068   
        4   2019-06-15 16:51:36.659252882 2019-06-15 16:34:30.590878248   
2       1   2019-06-15 14:44:33.601055145 2019-06-15 14:37:25.596413851   

                           fin_time_end95 personal_estimate_interval  
team_id leg                                                           
1       1   2019-06-15 14:49:27.716343880                  17.825123  
        2   2019-06-15 15:32:00.982809544                  13.262589  
        3   2019-06-15 16:22:42.894754171                  26.305104  
        4   2019-06-15 17:11:35.190454006                  12.640988  
2       1   2019-06-15 14:52:40.109992266                  15.241893  

[5 rows x 21 columns]

In [16]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

for_print[for_print['team'].str.contains("Stadin")]
#for_print = for_print[for_print['team'].str.contains("Puskasilimät")]

for_print = for_print[[
 'leg',
 'name', 
 'num_runs', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

,leg,name,num_runs,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,,
1,1,Tilda Östberg,1.0,31.64,39.47,49.46,17.83
1,2,Magdalena Olsson,7.0,34.43,40.40,47.69,13.26
1,3,Julia Jakob,1.0,32.33,43.47,58.63,26.31
1,4,Tove Alexandersson,7.0,41.70,47.62,54.34,12.64
2,1,Amanda Hansson,2.0,37.43,44.56,52.67,15.24
...,...,...,...,...,...,...,...
1722,4,Johanna Liinamaa,3.0,72.65,89.34,109.88,37.24
1723,1,Elina Holstikko,0.0,63.37,90.01,128.49,65.12
1723,2,Mirka Leinonen,0.0,59.90,85.19,121.63,61.73


In [17]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'num_runs',     
 'ind_log_mean', 
 'ind_log_std',
 'personal_start95', 
 'personal_median',
 'personal_end95',    
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Downt")]

,team_id,leg,team,name,num_runs,ind_log_mean,ind_log_std,personal_start95,personal_median,personal_end95,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
1332,341,1,Rasti-Jyry 1,Jola Maisala,8.0,2.188600,0.126500,41.790162,53.689520,68.899487,3.982588,0.127956,2019-06-15 11:41:47.409715652+00:00,2019-06-15 12:08:53.969198465+00:00,2019-06-15 11:53:41.371186972+00:00,0.0000,0.0
1333,341,2,Rasti-Jyry 1,Kaija Rantoja,7.0,2.296600,0.144100,42.653784,56.578240,74.827359,4.706925,0.097018,2019-06-15 12:31:38.752261400+00:00,2019-06-15 13:13:44.427448273+00:00,2019-06-15 12:50:40.165865898+00:00,0.0000,0.0
1334,341,3,Rasti-Jyry 1,Jenni Castren,5.0,2.334200,0.141700,57.318213,75.264186,99.733188,5.230327,0.081463,2019-06-15 13:39:10.474757671+00:00,2019-06-15 14:39:35.871567249+00:00,2019-06-15 14:06:41.159544706+00:00,0.0000,0.0
1335,341,4,Rasti-Jyry 1,Inka Lemmetyinen,1.0,2.293200,0.168000,56.194503,78.201619,108.995916,5.583907,0.075696,2019-06-15 14:49:48.295929432+00:00,2019-06-15 16:09:16.435700655+00:00,2019-06-15 15:25:58.567618847+00:00,0.0000,0.0
2076,528,1,Rasti-Jyry 2,Päivi Taponen,3.0,2.361600,0.167728,45.841543,64.010481,88.942752,4.156705,0.168836,2019-06-15 11:45:50.492583752+00:00,2019-06-15 12:28:56.565095425+00:00,2019-06-15 12:04:00.628861427+00:00,0.0000,0.0
2077,528,2,Rasti-Jyry 2,Mervi Luurila,2.0,2.473200,0.183049,46.938592,67.727465,97.035718,4.886987,0.126250,2019-06-15 12:43:31.769454479+00:00,2019-06-15 13:50:34.901306629+00:00,2019-06-15 13:12:29.277494669+00:00,0.0000,0.0
2078,528,3,Rasti-Jyry 2,Suvi Tiainen,2.0,2.646600,0.187243,70.880697,102.632327,148.715554,5.466279,0.109437,2019-06-15 14:11:24.054644108+00:00,2019-06-15 15:54:02.535903692+00:00,2019-06-15 14:56:35.179152966+00:00,0.0000,0.0
2079,528,4,Rasti-Jyry 2,Tiina Isoniemi,6.0,2.153800,0.067659,59.406170,68.056287,77.798333,5.720484,0.086310,2019-06-15 15:17:52.092338800+00:00,2019-06-15 17:02:49.031813860+00:00,2019-06-15 16:04:50.507595778+00:00,0.1176,0.0
2916,740,1,Rasti-Jyry 4,Mila Viksilä,0.0,2.281775,0.182322,41.377163,59.037191,84.290799,4.075804,0.181331,2019-06-15 11:41:22.629774809+00:00,2019-06-15 12:24:17.447962523+00:00,2019-06-15 11:59:02.231448412+00:00,0.0000,0.0
2917,740,2,Rasti-Jyry 4,Tiina Sidensnöre,7.0,2.262600,0.146600,41.019468,54.703134,72.976964,4.739506,0.118814,2019-06-15 12:30:42.065854549+00:00,2019-06-15 13:24:16.770614386+00:00,2019-06-15 12:54:24.057186842+00:00,0.0000,0.0


In [18]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [19]:
for_web.shape

(6728, 17)

In [20]:
for_print[['num_runs', 'personal_median', "personal_estimate_interval"]].groupby('num_runs').agg(["mean", "count"]).round(2)


personal_median       personal_estimate_interval      
                    mean count                       mean count
num_runs                                                       
0.0                81.96  1646                      65.16  1646
1.0                80.73  1021                      61.89  1021
2.0                81.43   730                      58.09   730
3.0                78.09   697                      55.94   697
4.0                76.82   557                      32.68   557
5.0                75.57   566                      35.25   566
6.0                73.00   551                      33.58   551
7.0                71.35   571                      32.76   571
8.0                71.38   386                      32.34   386
9.0                84.02     2                      35.82     2
10.0               62.46     1                      16.47     1

In [21]:
for_web["interval_length"] = for_web.fin_time_end95 - for_web.fin_time_start95
for_web["interval_length"].median()
#for_web[['leg', 'interval_length', "num_runs"]].info()

Timedelta('0 days 01:08:52.478938')

In [22]:
for_web[['leg', 'interval_length', "num_runs"]].groupby('leg').agg(["mean", "count"]).round(2)

num_runs      
        mean count
leg               
1       2.97  1682
2       2.75  1682
3       2.98  1682
4       3.23  1682